# MNIST database classification
This example is based on the work of quantum neural networks used as classifiers by Farhi et al (https://arxiv.org/abs/1802.06002).

A quantum and a classical neural network are compared on how they can classify pictures that represent two possible handwritten digits (a 3 and a 6).

In [2]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit


## Load the data
We upload the MNIST database from Keras

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))


11493376/11490434 [==============================] - 1s 0us/step
Number of original training examples: 60000
Number of original test examples: 10000
